In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install trl transformers accelerate git+https://github.com/huggingface/peft.git -Uqqq
!pip install datasets bitsandbytes einops wandb -Uqqq

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.7/79.7 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 89.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196

In [ ]:
%%capture
!pip install datasets

In [ ]:
import torch
from datasets import load_dataset
from datasets import DatasetDict, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, GenerationConfig
from peft import LoraConfig, get_peft_model, PeftConfig, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
import warnings
import pandas as pd
import json
from tqdm import tqdm

#train dataset

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
with open(f"/content/drive/MyDrive/shroom/val.model-agnostic.json", "r") as outfile:
  val_model_agnostic=json.load(outfile)
len(val_model_agnostic)

499

In [ ]:
val_model_agnostic_df=pd.DataFrame(data=val_model_agnostic, columns=["src", "tgt", "hyp", "ref", "task", "model", "labels", "label", "p(Hallucination)"])
val_model_agnostic_df.head(4)

,src,tgt,hyp,ref,task,model,labels,label,p(Hallucination)
0,The writer had just entered into his eighteent...,Resembling a weasel (in appearance).,Resembling or characteristic of a weasel.,tgt,DM,,"[Hallucination, Not Hallucination, Not Halluci...",Not Hallucination,0.2
1,Sailors ' and fishermen 's <define> sheath - k...,.,Alternative form of sheath knife,tgt,DM,,"[Hallucination, Hallucination, Hallucination, ...",Hallucination,0.8
2,"As to age , Bead could not form any clear impr...","(poetic) An instant, a short moment.",(obsolete) A short period of time.,tgt,DM,,"[Not Hallucination, Not Hallucination, Not Hal...",Not Hallucination,0.0
3,Because redpillers are usually normies or <def...,"(incel, _, slang) A man of a slightly lower ra...",(slang) An incel.,tgt,DM,,"[Not Hallucination, Not Hallucination, Halluci...",Not Hallucination,0.2


In [ ]:
with open(f"/content/drive/MyDrive/shroom/val.model-aware.v2.json", "r") as outfile:
  val_model_aware=json.load(outfile)
len(val_model_aware)

501

In [ ]:
val_model_aware_df=pd.DataFrame(data=val_model_aware, columns=["src", "tgt", "hyp", "ref", "task", "model", "labels", "label", "p(Hallucination)"])
val_model_aware_df.head(4)

,src,tgt,hyp,ref,task,model,labels,label,p(Hallucination)
0,The sides of the casket were covered with heav...,A decorative feature that sits on top of somet...,A sloping top .,tgt,DM,ltg/flan-t5-definition-en-base,"[Not Hallucination, Hallucination, Not Halluci...",Hallucination,0.6
1,Please try not to overreact if she drives badl...,To react too much or too intensely .,To react too much .,tgt,DM,ltg/flan-t5-definition-en-base,"[Not Hallucination, Not Hallucination, Not Hal...",Not Hallucination,0.0
2,"To prevent spoilage , store in a cool , dry pl...",The process of spoiling .,The process of spoiling ; the state of being s...,tgt,DM,ltg/flan-t5-definition-en-base,"[Hallucination, Not Hallucination, Hallucinati...",Hallucination,0.6
3,The way the opposition has framed the argument...,To construct in words so as to establish a con...,To arrange in a particular way .,tgt,DM,ltg/flan-t5-definition-en-base,"[Hallucination, Not Hallucination, Not Halluci...",Hallucination,0.6


In [ ]:
no_count=0

In [ ]:
texts=[]

for i in tqdm(range(len(val_model_aware_df))):
  sample=val_model_aware_df.iloc[i]

  hyp = str(sample['hyp'])
  src = str(sample['src'])
  tgt = str(sample['tgt'])
  task=sample['task']

  if task == "PG":
      context = f"Context: {src}"
  else: #i.e. task == "MT" or task == "DM":
      context = f"Context: {tgt}"

  sentence = f"Sentence: {hyp}"

  input_text=f"""
  <human>: {context}\n{sentence}\nIs the Sentence supported by the Context above? Answer using ONLY yes or no:
  """.strip()+"\n"

  if sample['label']=="Hallucination":
    label="no"
    no_count+=1
  else:
    label="yes"

  input_text+=f"<assistant>: [label]: {label}".strip()+"\n"

  texts.append(input_text)


100%|██████████| 501/501 [00:00<00:00, 9683.58it/s]


In [ ]:
for i in tqdm(range(len(val_model_agnostic_df))):
  sample=val_model_agnostic_df.iloc[i]

  hyp = str(sample['hyp'])
  src = str(sample['src'])
  tgt = str(sample['tgt'])
  task=sample['task']

  if task == "PG":
      context = f"Context: {src}"
  else: #i.e. task == "MT" or task == "DM":
      context = f"Context: {tgt}"

  sentence = f"Sentence: {hyp}"

  input_text=f"""
  <human>: {context}\n{sentence}\nIs the Sentence supported by the Context above? Answer using ONLY yes or no:
  """.strip()+"\n"

  if sample['label']=="Hallucination":
    label="no"
    no_count+=1
  else:
    label="yes"

  input_text+=f"<assistant>: [label]: {label}".strip()+"\n"

  texts.append(input_text)


100%|██████████| 499/499 [00:00<00:00, 10647.82it/s]


In [ ]:
d={
    'text':texts
}
df=pd.DataFrame(data=d)

In [ ]:
df

,text
0,<human>: Context: A decorative feature that si...
1,<human>: Context: To react too much or too int...
2,<human>: Context: The process of spoiling .\nS...
3,<human>: Context: To construct in words so as ...
4,<human>: Context: That in which one is concern...
...,...
995,<human>: Context: Tom walked over to Mary and ...
996,<human>: Context: I don't think Tom is going t...
997,<human>: Context: Russia had emerged as a seco...
998,<human>: Context: The children got some presen...


In [ ]:
df = df.sample(frac=1, random_state=42).reset_index(drop=False)
df

,index,text
0,521,"<human>: Context: (Geordie, informal) A friend..."
1,737,<human>: Context: I'll give it a shot.\nSenten...
2,740,<human>: Context: How come you're asking?\nSen...
3,660,<human>: Context: Serving to transport to anot...
4,411,<human>: Context: Israel demands an ongoing mi...
...,...,...
995,106,<human>: Context: To laugh boisterously .\nSen...
996,270,"<human>: Context: Indeed, it is on the agenda ..."
997,860,<human>: Context: Blimey!\nSentence: Nuclear B...
998,435,"<human>: Context: After the accident occurred,..."


In [ ]:
print(df.iloc[0]['text'])

<human>: Context: (Geordie, informal) A friend, pal, buddy, mate.
Sentence: (slang) The human body.
Is the Sentence supported by the Context above? Answer using ONLY yes or no:
<assistant>: [label]: no



In [ ]:
print(df.iloc[1]['text'])

<human>: Context: I'll give it a shot.
Sentence: I'll test it.
Is the Sentence supported by the Context above? Answer using ONLY yes or no:
<assistant>: [label]: yes



In [ ]:
print(df.iloc[105]['text'])

<human>: Context: (slang) A man who spends much of his free time working out at a gym.
Sentence: (informal) A man who works out at a gym.
Is the Sentence supported by the Context above? Answer using ONLY yes or no:
<assistant>: [label]: yes



In [ ]:
from datasets import DatasetDict, Dataset

data = DatasetDict({'train': Dataset.from_pandas(df)})
data

DatasetDict({
    train: Dataset({
        features: ['index', 'text'],
        num_rows: 1000
    })
})

In [ ]:
data["train"][0]

{'index': 521,
 'text': '<human>: Context: (Geordie, informal) A friend, pal, buddy, mate.\nSentence: (slang) The human body.\nIs the Sentence supported by the Context above? Answer using ONLY yes or no:\n<assistant>: [label]: no\n'}

In [ ]:
print(data["train"][0]['text'])

<human>: Context: (Geordie, informal) A friend, pal, buddy, mate.
Sentence: (slang) The human body.
Is the Sentence supported by the Context above? Answer using ONLY yes or no:
<assistant>: [label]: no



#training

In [ ]:
# model_name = "ybelkada/falcon-7b-sharded-bf16" # sharded falcon-7b model
model_name = "tiiuae/falcon-7b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,            # load model in 4-bit precision
    bnb_4bit_quant_type="nf4",    # pre-trained model should be quantized in 4-bit NF format
    bnb_4bit_use_double_quant=True, # Using double quantization as mentioned in QLoRA paper
    bnb_4bit_compute_dtype=torch.bfloat16, # During computation, pre-trained model should be loaded in BF16 format
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config, # Use bitsandbytes config
    device_map="auto",  # Specifying device_map="auto" so that HF Accelerate will determine which GPU to put each layer of the model on
    trust_remote_code=True, # Set trust_remote_code=True to use falcon-7b model with custom code
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True) # Set trust_remote_code=True
tokenizer.pad_token = tokenizer.eos_token # Setting pad_token same as eos_token

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
# model = prepare_model_for_kbit_training(model)
model= prepare_model_for_kbit_training(model, use_gradient_checkpointing=False)

lora_alpha = 32 # scaling factor for the weight matrices
lora_dropout = 0.05 # dropout probability of the LoRA layers
lora_rank = 32 # dimension of the low-rank matrices

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_rank,
    bias="none",  # setting to 'none' for only training weight params instead of biases
    task_type="CAUSAL_LM",
    target_modules=[         # Setting names of modules in falcon-7b model that we want to apply LoRA to
        "query_key_value",
        "dense",
        "dense_h_to_4h",
        "dense_4h_to_h",
    ]
)

peft_model = get_peft_model(model, peft_config)
print_trainable_parameters(model)

trainable params: 65273856 || all params: 3674018688 || trainable%: 1.776633750209166


In [ ]:
output_dir = "/content/LLM/falcon-7b-shroom"
per_device_train_batch_size = 2 # reduce batch size by 2x if out-of-memory error
gradient_accumulation_steps = 2  # increase gradient accumulation steps by 2x if batch size is reduced
optim = "paged_adamw_32bit" # activates the paging for better memory management
save_strategy="steps" # checkpoint save strategy to adopt during training
save_steps = 10 # number of updates steps before two checkpoint saves
logging_steps = 10  # number of update steps between two logs if logging_strategy="steps"
learning_rate = 2e-4  # learning rate for AdamW optimizer
max_grad_norm = 0.3 # maximum gradient norm (for gradient clipping)
warmup_ratio = 0.03 # number of steps used for a linear warmup from 0 to learning_rate
lr_scheduler_type = "cosine"  # learning rate scheduler

num_epochs=3

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    bf16=False,
    max_grad_norm=max_grad_norm,
    # max_steps=max_steps,
    num_train_epochs=num_epochs,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    push_to_hub=True,
    tf32=False,
    save_total_limit=2
)

In [ ]:
trainer = SFTTrainer(
    model=peft_model,
    train_dataset=data['train'],
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=1024,
    tokenizer=tokenizer,
    args=training_arguments,
)

for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
peft_model.config.use_cache = False
trainer.train()

wandb: Currently logged in as: zara-r-1990-r. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,3.089100
20,1.846100
30,0.978000
40,0.784500
50,0.675700
60,1.461100
70,1.137100
80,0.859400
90,0.697600
100,0.654000


TrainOutput(global_step=750, training_loss=0.753717750231425, metrics={'train_runtime': 3251.7372, 'train_samples_per_second': 0.923, 'train_steps_per_second': 0.231, 'total_flos': 7891075131949056.0, 'train_loss': 0.753717750231425, 'epoch': 3.0})

In [ ]:
PEFT_MODEL_path ="/content/drive/MyDrive/shroom/models/falcon_7b_shroom"
trainer.save_model(PEFT_MODEL_path)

events.out.tfevents.1706455269.be2ece349b63.22601.0:   0%|          | 0.00/17.5k [00:00<?, ?B/s]